In [62]:
import requests
import pandas as pd
from my_api import api_key

# https://data.mos.ru/ источник данных

#Получаем общее кол-во строк в наборе данных
total = requests.get(f'https://apidata.mos.ru/v1/datasets/61681/count?api_key={api_key}').json()

# Создаем пустой список, который будем наполнять запросами по 500 строк (500 - ограничение апи)
data_for_df = []
print(f'Всего {total} строк в датасете')

#циклом по 500 строк забираем весь датасет
i = 0
while i < total:
    url=f'https://apidata.mos.ru/v1/datasets/61681/rows?api_key={api_key}&$top=500&$skip={str(i)}'
    response = requests.get(url).json()
    data_for_df += response
    print(f'{i}-{i+499}', end='; ')
    i += 500

# загружаем датасет в пандас
df = pd.json_normalize(data_for_df)

Всего 7621 строк в датасете
0-499; 500-999; 1000-1499; 1500-1999; 2000-2499; 2500-2999; 3000-3499; 3500-3999; 4000-4499; 4500-4999; 5000-5499; 5500-5999; 6000-6499; 6500-6999; 7000-7499; 7500-7999; 

In [73]:
# убираю лишнее из названия колонок
for column in df.columns:
    df.rename(columns={column:column.replace('Cells.','')},inplace=True)
# разбиваю список информации о станции на две колнки
df['base_station_id'] = df['StationInfo'].apply(lambda x: x[0]['BaseStation'] if len(x)>0 else 'not defined')
df['operator'] = df['StationInfo'].apply(lambda x: x[0]['Operator'] if len(x)>0 else 'not defined')

# удаляю лишнее
del(df['StationInfo'])
del(df['geoData.coordinates'])
del(df['geoData.type'])


In [77]:
# смотрю кол-во станций
df[['operator','Number']].groupby('operator',as_index=False).count().sort_values('Number',ascending=False)

,operator,Number
2,ООО «Т2 Мобайл»,2188
3,ПАО «ВымпелКом»,2089
5,ПАО «Мобильные ТелеСистемы»,1909
4,ПАО «МегаФон»,1013
1,ООО «Антарес»,366
0,0,55
6,Юнисел,1


In [74]:
df.to_csv('base_stations.csv')